In [1]:
import numpy as np;
import pandas as pd;

from sklearn.svm import LinearSVC
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import matthews_corrcoef,make_scorer
mccScorer = make_scorer(matthews_corrcoef)
from numpy import mean
from numpy import std

from google.colab import drive
drive.mount("/content/gdrive")

from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif
from sklearn.model_selection import cross_validate

from sklearn.preprocessing import StandardScaler

Mounted at /content/gdrive


In [2]:
#read training dataset 1
data1 = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/Dataset_1_Training.csv')

X_train_actual_1=data1[:-2].T[1:].values

y_train_actual_1 = data1[-2:].T[-130:].T.values

In [3]:
#read training dataset 2
data2 = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/Dataset_2_Training.csv')

X_train_actual_2=data2[:-4].T[1:].values

y_train_actual_2 = data2[-4:].T[-340:].T.values

In [4]:
co1={'X':X_train_actual_1,'Y': y_train_actual_1[0]};
co2={'X':X_train_actual_1,'Y': y_train_actual_1[1]};

co3={'X':X_train_actual_2,'Y': y_train_actual_2[0]};
co4={'X':X_train_actual_2,'Y': y_train_actual_2[1]};
co5={'X':X_train_actual_2,'Y': y_train_actual_2[2]};
co6={'X':X_train_actual_2,'Y': y_train_actual_2[3]};

In [5]:
def feature_selector(X,Y,numberOfFeaturesToBeSelected=1):
  if(numberOfFeaturesToBeSelected == 1):
    totalNumberOfFeatures=X.shape[1]
    numberOfFeaturesToBeSelected=totalNumberOfFeatures//2;
  # define feature selection
  fs = SelectKBest(score_func=f_classif, k=numberOfFeaturesToBeSelected)
  # apply feature selection
  return fs.fit(X, Y)

In [6]:
def SVM_with_HPTuning(X,Y):
  scaler = StandardScaler()
  X_train_scaled = scaler.fit_transform(X.astype(np.float32))
  # define model and parameters
  pipeline = Pipeline(
    [
      ('selector',SelectKBest(f_classif)),
      ('model',LinearSVC())
    ]
  )
  penalty = ['l1', 'l2']
  loss = ['squared_hinge','hinge']
  C = [50,10,1.0, 0.1, 0.01]
  class_weight=['balanced','None']
  # define grid search
  grid = dict(model__penalty=penalty,model__loss=loss,model__C=C,model__class_weight=class_weight,selector__k=[int(x) for x in np.linspace(start = 5 , stop = X.shape[1]//4, num = 30)])
  cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=1, random_state=1)
  search = GridSearchCV(estimator=pipeline, param_grid=grid, cv=cv, scoring=mccScorer,verbose=2)
  # execute search
  result = search.fit(X_train_scaled, Y.astype('int'))
  # summarize result
  print('Best Score: %s' % result.best_score_)
  print('Best Hyperparameters: %s' % result.best_params_)

In [ ]:
SVM_with_HPTuning(**co1);

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
SVM_with_HPTuning(**co2);

Output hidden; open in https://colab.research.google.com to view.

In [7]:
SVM_with_HPTuning(**co3);

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
SVM_with_HPTuning(**co4);

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
SVM_with_HPTuning(**co6);